In [ ]:
import cv2
import matplotlib.pyplot as plt
# Add relative path
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from utils.augment_img import preprocess_img
%matplotlib inline


original_img = cv2.cvtColor(cv2.imread('/media/sharedHDD/LISA/aiua120214-2/frameAnnotations-DataLog02142012_002_external_camera.avi_annotations/signalAhead_1331866722.avi_image5.png'), cv2.COLOR_BGR2RGB)
img = cv2.resize(original_img, (608, 608))
# img = original_img
plt.figure(figsize=(14, 14))
plt.imshow(img)
print(img.shape)

In [ ]:
from model.densenet import DenseNet
from model.MobileYolo import MobileYolo
import tensorflow as tf
import keras.backend as K
from cfg import *
K.clear_session()  # to avoid duplicating model

with tf.device('/cpu:0'):
    densenet = DenseNet(img_input=(608, 608, 3), reduction=0.5, freeze_layers=True, dropout_rate=None)
    yolov2 = MobileYolo(feature_extractor=densenet, num_anchors=N_ANCHORS, num_classes=N_CLASSES, dropout=None,
                        fine_grain_layer='conv4_blk')
    yolov2.model.load_weights('../yolov2.weights')
    yolov2.model.summary()

In [ ]:
# Predict
with tf.Session() as sess:
    boxes_prediction, scores_prediction, classes_prediction = yolov2.predict(img)
    
print(boxes_prediction, scores_prediction, classes_prediction)

In [ ]:
from utils.draw_boxes import Box, draw
result = boxes_prediction
bboxes = []

orig_size = original_img.shape
for box, score, cls in zip(result, scores_prediction, classes_prediction):
    box = box * np.array(2 *[orig_size[0]/960., orig_size[1]/960.])
    y1, x1, y2, x2 = box
    print("Found a '{}' at {} with {:-2f}%".format(0, (x1, y1, x2, y2), 100*score))
    bboxes.append(Box(x1, y1, x2, y2, 0, score))

result = draw(original_img, bboxes)
plt.figure(figsize=(15, 15))

plt.imshow(result)
print(img.shape)